<font size = 5>Більше про відкриті дані: </font> 

https://docs.google.com/presentation/d/1t9ATBb3Y9YQDZA9ctpNVAVt2AVbncxqPAZRtjBZZ-GQ/edit?usp=sharing

<font size = 5>Набір даних:</font> 

https://data.gov.ua/dataset/06779371-308f-42d7-895e-5a39833375f0

<font size = 5>__Що ми можемо дізнатись__:</font> 
    <ul>
     <li>топ реєстрацій автомобілів за маркою;</li>
        <li>топ реєстрацій автомобілів за моделлю</li>
        <li>кількість реєстрацій авто по країні</li>
        <li>кількість реєстрацій авто по областях</li>
    </ul>

<font size = 5>__Імпортуємо бібілотеки__ </font>

In [ ]:
import requests # робимо запити по url
import json # для роботи із json файлами
import pandas as pd # основна бібілотека для роботи із даними
import numpy as np # для роботи із матрицями, веторами та масивами
from matplotlib import pyplot as plt # бібліотека для побудови графіків
from pprint import pprint as pp # бібліотека для людиночитаного представлення даних 
import re # бібліотека для робот із регулярними виразами
import seaborn as sns # надбудова для спрошення побудови графіків
import csv # бібліотека для роботи із csv файлами
import io # модуль із командами для взаємодіїі із різними типами файлів
import zipfile # бібліотека для роботи із .zip файлами
import os # бібліотека для взаємодії із системою
from tqdm import tqdm # звичайний прогрес бар

# Працюємо із АРІ порталу

<font size = 5>__Робимо запит до АРІ Data.gov.ua__</font>

__документація по роботі із АРІ Порталу:__ https://data.gov.ua/pages/aboutuser2

__ідентифікаційний номер набору даних по реєстрації авто в Україні:__ 	06779371-308f-42d7-895e-5a39833375f0

__посилання на набір даних:__ https://data.gov.ua/dataset/06779371-308f-42d7-895e-5a39833375f0

In [ ]:
response = requests.get("https://data.gov.ua/api/3/action/package_show?id=06779371-308f-42d7-895e-5a39833375f0").json()

In [ ]:
# звичайний метод не надто ефективний в такому випадку
print(response)

In [ ]:
# краще спрацьовує модуль pprint https://docs.python.org/3/library/pprint.html
pp(response) #переглядаємо вміст відповіді

Нам треба знайти дані із атрибутом "url_redirected_to" повний шлях: "["result"]["resources"][_порядковий номер ресурсу_]["url"]"
Отримуємо дані за допомогою циклу, яким перебираємо всі посилання на ресурси у наборі.

In [ ]:
for i in response["result"]["resources"]:
    print(i["description"], " :", i["url"]) # можна використовувати +, але протыше використовувати кому

In [ ]:
# циклом збираємо url адреси
urls = [i["url"] for i in response["result"]["resources"]]
urls

In [ ]:
# створюємо папку "data", якщо папка існує отримаємо помилку
os.mkdir("data")

In [ ]:
# за допомогою запитів збираємо архіви розпакрвуємо їх та складаємо в папку
for url in urls:
    response = requests.get(url)
    zip_data = io.BytesIO(response.content)
    # розпаковуємо архів із даними
    print("zip_data = io.BytesIO(rs.content)")
    with zipfile.ZipFile(zip_data) as zip_file:
        print("zip_file.extractall")
        zip_file.extractall("data")

Отримуємо перелік всіх файлів в директорії

In [ ]:
# складаємо перелік файлів в змінну та викликаємо її
directory = os.listdir("data")
directory

# Підвантажєння та підготовка даних

## Перекодовуємо інформацію про області та сервісні центри

In [ ]:
# тестуємо завантаження даних
# вимикаємо сповіщення про помилки та примусово зазначаємо роздільник
# досліджуємо датасет
data = pd.read_csv("data/tz_opendata_z01012013_po31122013.csv", error_bad_lines = False, sep = ";")
data

In [ ]:
# циклом завантажуємо кожен окремий файл та додаємо його до датасету - pd.concat
data_wrong = pd.DataFrame()
for f in tqdm(directory):
    table = pd.read_csv("data/"+ str(f), error_bad_lines = False, sep = ";")
    data_wrong= pd.concat([data_wrong, table])
data_wrong

In [ ]:
# якщо переглянемо колонки то побачимо, що через різний регістр назв ми отримали неправильний масив
data_wrong.columns

In [ ]:
# правильний спосіб завантаження даних в pandas, якщов они зберігаються у кількох таблицях
data_right = pd.DataFrame()
for f in tqdm(directory):
    table = pd.read_csv("data/"+ str(f), error_bad_lines = False, sep = ";")
    table.columns = table.columns.str.lower() #у деяких старих наборах даних заголовки колонок великими літерами
    data_right = pd.concat([data_right, table])
data_right

In [ ]:
data_right.columns

In [ ]:
data = data_right

In [ ]:
# очищуємо зайві перемінні для економії пам'яті
del data_right
del data_wrong

Дані із кодами КОАТУУ - частково не заповнені, ми не хочемо втратити інформацію про жодну зареєстровану дію, тому ми не будемо використовувати дані із атрибуту data["reg_addr_koatuu"], натомість ми геокодуємо дані з інформації про орган, який здійснював реєстраційні дії. Але ці дані доволі брудні, також під час формування набору у 2013 - 2015 роках проховодилась реформа, що призвело до того, що ми маємо інформацію про різні органи реєстрації. Сучасні органи кодуються у форматі ТСЦ ххуу - де хх - код регіону, а уу - унікальний код сервісного центру. 
Завантажимо таблицю, яка містить інформацію про сучасну прив'язку ТСЦ до областей та виведемо її, для цього треба налаштувати Pandas

In [ ]:
# встановлення обмеження на кількість відображених рядків
pd.set_option('display.max_rows', 160)
 
# скидання обмеження на кількість відображених атрибутів
pd.set_option('display.max_columns', None)
 
# встановлення обмеження на кількість відображених символів в комірці
pd.set_option('display.max_colwidth', 25)

In [ ]:
# читаємо таблицю
deps = pd.read_excel("dep.xlsx")

In [ ]:
# виводимо її на екран для ознайомлення
deps

In [ ]:
# дублюємо колонку із даними про департаменти для подальшого кодування регіонів
data["region"] = data["dep"]

In [ ]:
# зводимо все до нижнього регістру
data["region"] = data["region"].str.lower()

# стандартизуємо дані із назвами сервісних центрів

data["region"] = data["region"].str.replace(r"\((\d{4})\).*", r"тсц \1", regex=True)
# Для быльшої зрозумілості виокремити області вирішив через окремі ітерації
data["region"] = data["region"].str.replace(r".*макарі.*|.*київсь.*", r"київська", regex=True)
data["region"] = data["region"].str.replace(r".*чернівець.*", r"чернівецька", regex=True)
data["region"] = data["region"].str.replace(r".*херсон.*", r"херсонська", regex=True)
data["region"] = data["region"].str.replace(r".*мог-подільськ.*|.*липовец.*|.*ямпіл.*|.*калинівс.*|.*тульчин.*|.*козятин.*|.*вінниць.*|.*жмерин.*|.*берша.*|.*крижо.*|.*хмільн.*|.*гайсин.*|.*немирів.*", r"вінницька", regex=True)
data["region"] = data["region"].str.replace(r".*донець.*", r"донецька", regex=True)
data["region"] = data["region"].str.replace(r".*тернопіль.*", r"тернопільська", regex=True)
data["region"] = data["region"].str.replace(r".*запорі.*", r"запорізька", regex=True)
data["region"] = data["region"].str.replace(r".*волин.*|.*горохів.*|.*луцьк.*|.*ковел.*|.*ківер.*|.*любом.*|.*кашир.*", r"волинська", regex=True)
data["region"] = data["region"].str.replace(r".*дніпропе.*", r"дніпропетровська", regex=True)
data["region"] = data["region"].str.replace(r".*полтав.*", r"полтавська", regex=True)
data["region"] = data["region"].str.replace(r".*чернігів.*", r"чернігівська", regex=True)
data["region"] = data["region"].str.replace(r".*миколаїв.*", r"миколаївська", regex=True)
data["region"] = data["region"].str.replace(r".*житомир.*|.*овруц.*|.*овруч.*|.*коростен.*|.*чудні.*|.*попільня.*|.*бердичі.*|.*радомиш.*", r"житомирська", regex=True)
data["region"] = data["region"].str.replace(r".*львів.*", r"львівська", regex=True)
data["region"] = data["region"].str.replace(r".*черкас.*", r"черкаська", regex=True)
data["region"] = data["region"].str.replace(r".*івано.*", r"івано-Франківська", regex=True)
data["region"] = data["region"].str.replace(r".*харків.*", r"харківська", regex=True)
data["region"] = data["region"].str.replace(r".*рівне.*", r"рівненська", regex=True)
data["region"] = data["region"].str.replace(r".*сум.*", r"сумська", regex=True)
data["region"] = data["region"].str.replace(r".*нетіши.*", r"хмельницька", regex=True)
data["region"] = data["region"].str.replace(r".*іршав.*|.*міжгірсь.*|.*виноградів.*|.*тячів.*|.*сваляв.*|.*берегів.*|.*берегов.*|.*рахів.*|.*ужгород.*|.*мукаче.*|.*мукачів.*|.*хуст.*", r"закарпатська", regex=True)
data["region"] = data["region"].str.replace(r".*м\. кіровсь.*|.*луганс.*|.*лутуг.*", r"луганська", regex=True)
data["region"] = data["region"].str.replace(r".*м\. київ.*|.*києв.*|.*головний.*|.*міністерство.*", r"київ", regex=True)
data["region"] = data["region"].str.replace(r".*крим.*|.*євпаторія.*|.*судак.*|.*армян.*|.*сімферополь.*|.*симферопіль.*|.*бахчисарай.*", r"крим", regex=True)
data["region"] = data["region"].str.replace(r"центр|тсц даі", r"тсц", regex=True)

In [ ]:
# перевіряємо унікальні значення після використання регулярних виразів
pp(data["region"].unique())

Кодуємо області в тіх записах де відомі номери сервісних центрів

In [ ]:
data["region"] = data["region"].str.lower()
data["region"] = data["region"].str.replace(r"тсц 05.*", r"вінницька", regex=True)
data["region"] = data["region"].str.replace(r"тсц 07.*", r"волинська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 12.*", r"дніпропетровська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 14.*|тсц даі 14.*", r"донецька", regex=True)
data["region"] = data["region"].str.replace(r"тсц 18.*", r"житомирська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 21.*", r"закарпатська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 23.*", r"запорізька", regex=True)
data["region"] = data["region"].str.replace(r"тсц 26.*", r"івано-Франківська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 32.*", r"київська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 35.*", r"кіровоградська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 44.*", r"луганська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 46.*", r"львівська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 48.*", r"миколаївська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 51.*", r"одеська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 53.*", r"полтавська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 56.*", r"рівненська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 59.*", r"сумська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 61.*", r"тернопільська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 63.*", r"харківська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 65.*", r"херсонська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 68.*", r"хмельницька", regex=True)
data["region"] = data["region"].str.replace(r"тсц 71.*", r"черкаська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 73.*", r"чернівецька", regex=True)
data["region"] = data["region"].str.replace(r"тсц 74.*|тсц даі 74.*", r"чернігівська", regex=True)
data["region"] = data["region"].str.replace(r"тсц 80.*", r"київ", regex=True)
data["region"] = data["region"].str.replace(r"тсц 85.*|тсц 01.*", r"крим", regex=True) # кодуємо дані спільно для Криму та Сімферополя
data["region"] = data["region"].str.replace(r"мвс україни.*", r"київ", regex=True) # кодуємо в останню чергу для уникнення помилок

In [ ]:
data["region"] = data["region"].str.replace(r".*кіровоград.*", r"кіровоградcька", regex=True)
data["region"] = data["region"].str.replace(r".*old_ддаі київ.*", r"київ", regex=True)

In [ ]:
data["region"] = data["region"].str.lower()

In [ ]:
# Перевіряємо чи все ок із регіонами
data["region"].unique()

In [ ]:
# Зберігаємо дані у файл
data.to_csv("data_normalize.csv")

In [ ]:
# у випадку, якщо щось не вантажиться
del data

## Підвантажуємо "чистий" набір із даними

In [ ]:
# Завантажуємо дані із файлу
data = pd.read_csv("data_normalize.csv")

In [ ]:
del data["Unnamed: 0"]

In [ ]:
data.columns

In [ ]:
data.head()

Визначаємо пропущені значення - будуємо теплокарту пропущених значень. 

In [ ]:
cols = data.columns
# визначаємо кольори
# жовтий - пропущені значення, синій - дійсні значення
colours = ['#000099', '#ffff00'] 
sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colours))

Переглядаємо статистику значень

In [ ]:
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Будуємо гістограму пропущених значень

In [ ]:
# будуємо індикатор для пропущених значень
for col in data.columns:
    missing = data[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  
        print('created missing indicator for: {}'.format(col))
        data['{}_ismissing'.format(col)] = missing


# будуємо гістограму на основі індикатора
ismissing_cols = [col for col in data.columns if 'ismissing' in col]
data['num_missing'] = data[ismissing_cols].sum(axis=1)

data['num_missing'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index', y='num_missing')

In [ ]:
del data["reg_addr_koatuu_ismissing"]
del data["fuel_ismissing"]
del data["capacity_ismissing"]
del data["own_weight_ismissing"]
del data["total_weight_ismissing"]
del data["n_reg_new_ismissing"]
del data["vin_ismissing"]
del data["num_missing"]

Продовжуємо досліджувати дані

In [ ]:
#Перевіряємо види реєстрацій автомобілів
data["oper_name"].unique()

In [ ]:
# видаляємо непотрібні атрибути
del data["reg_addr_koatuu"]
del data["oper_name"]
del data["dep_code"]
del data["dep"]
del data["capacity"]
del data["own_weight"]
del data["total_weight"]
del data["n_reg_new"]

In [ ]:
data.head()

In [ ]:
# завантажуємо словник із інформацією про реєстраційні дії
oper_name_dict = pd.read_csv("oper_name.csv", sep = ";",encoding = "cp1251" )
oper_name_dict.head()

In [ ]:
oper_name_dict

In [ ]:
# додаємо очищені дані про реєстраційні дії до основного датасету
oper_name_dict["oper_code"] = oper_name_dict["oper_code"].astype(str)
data["oper_code"] = data["oper_code"].astype(str)
data = pd.merge(data,oper_name_dict, on='oper_code',how='left')
data

In [ ]:
# створюємо новий стовпець із глобальним статусом
data["oper_name_new"] = data["oper_name_new"].str.replace(r'.*ЗНЯТТЯ.*', r"ЗНЯТТЯ", regex = True)
data["oper_name_new"] = data["oper_name_new"].str.replace(r'.*ПЕРВИННА.*', r"ПЕРВИННА", regex = True)

In [ ]:
data.head()

# Аналіз

Завантажуємо дані пілся OpenRefine
та проводимо left join many-to-one

In [ ]:
brand_splited = pd.read_excel("brand_splited.xlsx")
brand_splited = brand_splited[["brand", "brand_new"]]
brand_splited = brand_splited.set_index("brand")
brand_splited

In [ ]:
data = pd.merge(data,brand_splited, on='brand',how='left')

In [ ]:
data.head()

### топ __нових__ реєстрацій автомобілів за маркою

In [ ]:
# скидаємо індекс для того, щоб отримати унікальний ідентифікатор
data.reset_index(inplace = True)

In [ ]:
# робимо сабсет даних для візуалізації
barplot_brands = data[["brand_new","oper_code","person", "kind", "make_year"]]
barplot_brands = barplot_brands.loc[
    barplot_brands["oper_code"].isin(["30", "100", "11", "20", "10", "120", "99", "172", "105", "17"])] # фільтруємо ті реєстраційні операції, які нам потрібні

barplot_brands = barplot_brands.loc[
    barplot_brands["person"] == "P"] # вибираємо тип покупця, в даному випадку фізичні особи

barplot_brands = barplot_brands.loc[
    barplot_brands["kind"] == "ЛЕГКОВИЙ"] # вибираємо тип автомобіля

barplot_brands = barplot_brands.loc[
    barplot_brands["make_year"] >= 2000] # обираємо рік випуску автомобіля

# робимо зведену таблицю для простоти візуалізації
barplot_brands = pd.pivot_table(barplot_brands, index = "brand_new", aggfunc="count")
barplot_brands = barplot_brands.sort_values("oper_code", ascending = False) #сортуємо дані в порядку спадання

In [ ]:
barplot_brands.to_csv("barplot_brands.csv")

Посилання на документацію https://seaborn.pydata.org/generated/seaborn.barplot.html

In [ ]:
# індекс, який ми отримали через сводну таблицю, для того щоб побудувати графік (при зміні налаштувань графіка застосовується лише раз)
barplot_brands.reset_index(inplace = True)

In [ ]:
barplot = sns.barplot(
    data=barplot_brands.iloc[:19], # Кількість записів у ТОП, можемо редагувати під час оголошення графіку
    x ="oper_code",
    y = "brand_new",
    color="#0b7fab",
)

sns.set_theme(
    style = "white",
    rc={'figure.figsize':(15,11)}
)

barplot.set(xlabel ='кількість реєстрацій', ylabel='')
barplot.set(title = "Рейтинг найпопулярніших марок авто в Україні")
barplot.figure.set_figwidth(20) #<-- зміна розміру відображення графіку
barplot.figure.set_figheight(10)

### топ __нових__ реєстрацій автомобілв за моделлю фізичні особи

In [ ]:
# об'єднуємо стовпчик "марка" та "модель" для отримання унікальних записів
data["brand_model"] = data[["brand_new", "model"]].apply(tuple, axis=1)
data.head()

In [ ]:
model_j = pd.pivot_table(data, index = "brand_model", aggfunc="count")
model_j = model_j.sort_values("body", ascending = False)
model_j.head()

In [ ]:
# треба скинути індекс, який ми отримали через сводну таблицю, для того щоб побудувати графік (при зміні налаштувань графіка застосовується лише раз)
model_j.reset_index(inplace = True)

In [ ]:
barplot = sns.barplot(
    data=model_j.iloc[:19], # Кількість записів у ТОП, можемо редагувати під час оголошення графіку
    x="body",
    y = "brand_model",
    color="#0b7fab",
)

sns.set_theme(
    style = "white",
    rc={'figure.figsize':(15,11)}
)

barplot.set(title = "Рейтинг найпопулярніших моделей авто в Україні")
barplot.set(xlabel ='Кількість реєстрацій', ylabel='')

### Порівняння на лайнчарті кількості __нових__ реєстрацій АВТО за весь час з 2013 року

In [ ]:
linechart_registrations = data[["brand_new","oper_code","person", "kind", "d_reg"]]
linechart_registrations["oper_code"] = linechart_registrations["oper_code"].astype(str)
linechart_registrations["d_reg"] = pd.to_datetime(linechart_registrations["d_reg"])
linechart_registrations = linechart_registrations.loc[
    linechart_registrations["oper_code"].isin(["30", "100", "11", "20", "10", "99", "172", "105", "17"])] # фільтруємо ті реєстраційні операції, які нам потрібні

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["person"] == "P"] # вибираємо тип покупця, в даному випадку фізичні особи

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["kind"] == "ЛЕГКОВИЙ"] # вибираємо тип автомобіля

linechart_registrations = pd.pivot_table(linechart_registrations, index = "d_reg",aggfunc="count" )
#linechart_registrations['cum_sum'] = linechart_registrations['brand_new'].cumsum()
linechart_registrations.reset_index(inplace = True)

In [ ]:
# агрегуємо інформацію по місяцях
linechart_registrations["y_m"] = pd.to_datetime(linechart_registrations["d_reg"]).dt.strftime("%Y-%m")
linechart_registrations["y_m"] = pd.to_datetime(linechart_registrations["y_m"])
linechart_registrations

In [ ]:
lines = sns.lineplot(data=linechart_registrations, x = "y_m", y = "brand_new", linewidth=2.5)
lines.set(title = "кількість нових реєстрацій авто ")
lines.set(xlabel ='', ylabel='')
lines.figure.set_figwidth(20) #<-- зміна розміру відображення графіку
lines.figure.set_figheight(10)

### Порівняння на лайнчарті кількості __нових__ реєстрацій АВТО за весь час з 2013 року (варіант 2)

In [ ]:
# створ.ємо зведену таблицю по місяцями
linechart_registrations = pd.pivot_table(linechart_registrations, index = "y_m",aggfunc="count" )

In [ ]:
# скидаємо індекс
linechart_registrations.reset_index(inplace = True)

In [ ]:
linechart_registrations

In [ ]:
lines = sns.lineplot(data=linechart_registrations, x = "y_m", y = "brand_new", linewidth=2.5)
lines.set(title = "кількість нових реєстрацій авто ")
lines.set(xlabel ='', ylabel='')

### Порівняння кількості реєстрацій та кількості зняття з реєстрації

In [ ]:
linechart_registrations = data[["brand_new","oper_code", "person", "kind", "d_reg", "oper_name_new"]] # створюємо копію із датасету з потрібними полями 

linechart_registrations["oper_code"] = linechart_registrations["oper_code"].astype(str) # змінюємо тип коду реєстрації на текстовий

linechart_registrations["oper_name_new"] = linechart_registrations["oper_name_new"].astype(str) # змінюємо тип коду реєстрації на текстовий

linechart_registrations["d_reg"] = pd.to_datetime(linechart_registrations["d_reg"]) # перетвлрюємо дату реєстрації на дату
linechart_registrations = linechart_registrations.loc[
    linechart_registrations["oper_code"].isin([
        "30", "100", "11", "20", "10", "99",
        "172", "105", "17", "552", "553", "53", "536",
        "542", "570", "500", "554", "580", "560", "510",
        "535", "534", "540", "550", "561", "520", "532",
        "531", "533", "530"])] # фільтруємо ті реєстраційні операції, які нам потрібні

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["person"] == "P"] # вибираємо тип покупця, в даному випадку фізичні особи

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["kind"] == "ЛЕГКОВИЙ"] # вибираємо тип автомобіля

linechart_registrations = pd.pivot_table(linechart_registrations, index = ["oper_name_new", "d_reg"] ,aggfunc="count" ) # створюємо зведену таблицю
linechart_registrations['cum_sum'] = linechart_registrations['brand_new'].cumsum() # створюємо стовпич із кумулятивною сумою
linechart_registrations.reset_index(inplace = True) # прибераємо індекс


In [ ]:
linechart_registrations

In [ ]:
lines = sns.lineplot(data=linechart_registrations, x = "d_reg", y = "cum_sum", linewidth=2.5, hue="oper_name_new")
lines.set(title = "кількість нових реєстрацій авто ")
lines.set(xlabel ='', ylabel='')

### Сітка лайн чартів із порівнянням кількості __нових__ реєстрацій по областях

In [ ]:
# Create a dataset with many short random walks
linechart_registrations = data[["brand_new","oper_code","person", "kind", "d_reg", "region"]]
linechart_registrations["oper_code"] = linechart_registrations["oper_code"].astype(str)
linechart_registrations["d_reg"] = pd.to_datetime(linechart_registrations["d_reg"])

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["oper_code"].isin(["30", "100", "11", "20", "10", "120", "99"])] # фільтруємо ті реєстраційні операції, які нам потрібні

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["person"] == "P"] # вибираємо тип покупця, в даному випадку фізичні особи

linechart_registrations = linechart_registrations.loc[
    linechart_registrations["kind"] == "ЛЕГКОВИЙ"] # вибираємо тип автомобіля

linechart_registrations = pd.pivot_table(linechart_registrations, index = ["d_reg", "region"],aggfunc="count" )
linechart_registrations

In [ ]:
# треба скинути індекс, який ми отримали через сводну таблицю, для того щоб побудувати графік (при зміні налаштувань графіка застосовується лише раз)
linechart_registrations.reset_index(inplace = True)
linechart_registrations

In [ ]:
# агрегуємо інформацію по місяцях
linechart_registrations["y_m"] = pd.to_datetime(linechart_registrations["d_reg"]).dt.strftime("%Y-%m")
linechart_registrations["y_m"] = pd.to_datetime(linechart_registrations["y_m"])
linechart_registrations

In [ ]:
# ініціюємо сітку графіків для кожного регіону
grid = sns.FacetGrid(linechart_registrations, col="region", col_wrap=5, height=3)

grid.map(plt.plot, "y_m", "brand_new")
grid.set(xlabel ='', ylabel='')

In [ ]:
#Аналіз даних про вінкоди автомобілів
vin = data[data["vin"].notna()]
vin

In [ ]:
vin = pd.pivot_table(vin[["body", "vin"]], index = "vin", aggfunc = "count")
vin.reset_index(inplace = True)
vin = vin.sort_values(by = "body", ascending = False)
vin

In [ ]:
vin = vin[vin["body"] > 5]
vin

In [ ]:
# VTHHUNTERTG057735 - хтось затиснув "не ту" клавішу

data[data["vin"] == "YS2K6X20001869015"].sort_values(by = "d_reg")

In [ ]:
data[data["brand"] == "TESLA"]